In [1]:
import pandas as pd

In [32]:
weather = pd.read_csv('../assets/weather.csv')
train = pd.read_csv('../assets/train.csv')
test = pd.read_csv('../assets/test.csv')
spray = pd.read_csv('../assets/spray.csv')

### Columns with missing values:


#### Weather:

Tavg, Depart, WetBulb, Heat, Cool, PrecipTotal, StnPressure, SeaLevel, AvgSpeed

#### Train and Test:

None

### Weather

In [3]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [4]:
weather.dtypes

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
Depart          object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
Depth           object
Water1          object
SnowFall        object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object

Station is ID number.  Leave as integer.
Tmax and Tmin can be left as integers

In [5]:
weather.Date = pd.to_datetime(weather.Date)

In [7]:
# Change Tavg, Depart, WetBulb, Heat, Cool, PrecipTotal, StnPressure, SeaLevel, AvgSpeed to numeric
weather.Tavg = weather.Tavg.apply(lambda x: int(x) if str(x).isdigit() else None)
weather.Depart = weather.Depart.apply(lambda x: int(x) if str(x).isdigit() else None)
weather.WetBulb = weather.WetBulb.apply(lambda x: int(x) if str(x).isdigit() else None)
weather.Heat = weather.Heat.apply(lambda x: int(x) if str(x).isdigit() else None)
weather.Cool = weather.Cool.apply(lambda x: int(x) if str(x).isdigit() else None)
weather.PrecipTotal = weather.PrecipTotal.apply(lambda x: float(x) if str(x)[0].isdigit() else None)
weather.StnPressure = weather.StnPressure.apply(lambda x: float(x) if str(x)[0].isdigit() else None)
weather.SeaLevel = weather.SeaLevel.apply(lambda x: float(x) if str(x)[0].isdigit() else None)
weather.AvgSpeed = weather.AvgSpeed.apply(lambda x: float(x) if str(x)[0].isdigit() else None)

In [8]:
# Drop Sunrise, Sunset, CodeSum, Depth, Water1
# Sunrise, Sunset could be converted to datetimes but we won't be using them
# CodeSum is code for certain weather conditions, mostly empty
# Depth is always 0 for station 1, missing for station 0.  Adds no new information
# Water1 is 'M' for all values

weather = weather.drop(['Sunrise', 'Sunset', 'CodeSum', 'Depth', 'Water1'], axis=1)

In [9]:
# SnowFall is 0 or missing for most values.
# That makes sense since data is only available May through October of each year
# The only values with snow are 12 days with 'T' for trace, and one day with 0.1 inches
# Change SnowFall to categorical: 0 if no snow/missing, 1 if any snow

def snowcode(x):
    if x == 'M' or x =='0.0':
        return 0
    else: return 1
    
weather.SnowFall = weather.SnowFall.apply(snowcode)

In [10]:
weather.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                  float64
Depart                float64
DewPoint                int64
WetBulb               float64
Heat                  float64
Cool                  float64
SnowFall                int64
PrecipTotal           float64
StnPressure           float64
SeaLevel              float64
ResultSpeed           float64
ResultDir               int64
AvgSpeed              float64
dtype: object

### Train

In [14]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [23]:
train.dtypes

Date                      datetime64[ns]
Address                           object
Species                            int64
Block                              int64
Street                            object
Trap                              object
AddressNumberAndStreet            object
Latitude                         float64
Longitude                        float64
AddressAccuracy                    int64
NumMosquitos                       int64
WnvPresent                         int64
dtype: object

In [17]:
train['Date'] = pd.to_datetime(train['Date'])

In [28]:
# Drop Address, Block, Street

train = train.drop(['Address', 'Block', 'Street', 'AddressNumberAndStreet'], axis=1)

In [29]:
# Encode labels for Species, Trap
from sklearn.preprocessing import LabelEncoder

train.Species = LabelEncoder().fit_transform(train.Species)
train.Trap = LabelEncoder().fit_transform(train.Trap)

In [30]:
train.dtypes

Date               datetime64[ns]
Species                     int64
Trap                        int64
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object

### Test
#### Follow procedure for train

In [34]:
test['Date'] = pd.to_datetime(test['Date'])
test = test.drop(['Address', 'Block', 'Street', 'AddressNumberAndStreet'], axis=1)
test.Species = LabelEncoder().fit_transform(test.Species)
test.Trap = LabelEncoder().fit_transform(test.Trap)

In [35]:
test.dtypes

Id                          int64
Date               datetime64[ns]
Species                     int64
Trap                        int64
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
dtype: object

### Spray

In [38]:
spray.dtypes

Date          object
Time          object
Latitude     float64
Longitude    float64
dtype: object

In [40]:
# Convert Date to Datetime
spray.Date = pd.to_datetime(spray.Date)

# Drop Time
spray = spray.drop('Time', axis = 1)

In [41]:
spray.dtypes

Date         datetime64[ns]
Latitude            float64
Longitude           float64
dtype: object